In [1]:
import numpy as np
import pandas as pd

In [2]:
tr_act = pd.read_csv('../train/train_activity.csv')
te_act = pd.read_csv('../test/test_activity.csv')
tr_trade = pd.read_csv('../train/train_trade.csv')
te_trade = pd.read_csv('../test/test_trade.csv')
tr_payment = pd.read_csv('../train/train_payment.csv')
te_payment = pd.read_csv('../test/test_payment.csv')

In [3]:
tr_act.head()

,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt
0,7,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,4,2.088814,4.405057,-0.245477,4.454059,3.009797,-0.306354,1.184011,...,-0.03313,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,3.466939,-0.027758,-0.365536
1,8,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,5,2.673460,4.760178,-0.222527,6.107479,4.758528,-0.306354,1.709534,...,-0.03313,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,5.070359,-0.027758,-0.365536
2,3,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,-0.649522,-0.231021,-0.290803,-0.245978,-0.487666,-0.306354,-0.566706,...,-0.03313,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.498275,-0.027758,-0.365536
3,4,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,-0.658235,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.03313,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536
4,5,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,4,-0.654734,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.03313,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536


In [4]:
tr_act.columns

Index(['wk', 'acc_id', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt'],
      dtype='object')

# 복원 방법

데이터를 자유롭게 파악하기 위해서는 실제값이 있는 것이 좋다.  
그렇기에 각각의 설명변수에 대해 실제값 복원을 실시한다. 모든 변수에 대해 해당 작업 반복수행.  

name = '설명변수'  
$min0$와 $min1$은 정규화된 전체 데이터중에서 가장 작은값과 두번째로 작은 값이다.  
$r\_min0$와 $r\_min1$은 실제 게임에서 min0, min1의 정수형 값이다.  
실제값의 정보는 게임 플레이 경험, 파워북으로부터 얻었다.  

$s$(표준편차) = $\frac{r\_min1 - r\_min0}{min1 - min0}$  
$m$(평균) = $r\_min0 - s \times min0$  
<br><br><br><br>  
아래 'quest_exp'의 경우 min0의 r_min은 0 이고 min1의 r_min은 70 이므로  

$s = \frac{70}{min1 - min0}$  
$m = r\_min0 - s \times min0$  

이다.

### 원본 설명변수 = 정규화된 설명변수 * s + m  
train 데이터와 test 데이터는 같은 표준편차와 평균을 가지므로 그대로 계산한다.

In [5]:
name = 'quest_exp'
min0, min1 = sorted(tr_act[name].unique())[:2]
print("0: {}\n1: {}".format(min0, min1))

r_min0, r_min1 = 0, 70
s = (r_min1 - r_min0) / (min1 - min0)
m = r_min0 - s * min0
print("s: {}\nm: {}".format(s, m))

# 정규화된 설명변수에 s를 곱하고 m을 더하면 복원이 된다.
tr_sample = tr_act[name]*s + m
sorted(tr_sample.unique(), reverse=False)[:10]

0: -0.250423355452758
1: -0.25038721264045
s: 1936761.295813056
m: 485010.2624085371


[0.0,
 70.0,
 79.99999999965075,
 90.9999999992433,
 104.00000000011642,
 130.00000000017462,
 140.0,
 200.00000000017462,
 208.00000000034925,
 210.00000000180444]

In [6]:
# 설명변수 = (r_min0, r_min1)
counter = {
    "else": {"min": (0, 1)},
    "npc_hongmun": {"min": (0, 6)},
    "quest_exp": {"min": (0, 70)},
    "quest_hongmun": {"min": (0, 70)},
    "item_hongmun": {"min": (0, 10000)}
}

In [7]:
not_denormal = ["wk", "acc_id", "cnt_dt"]
for column in tr_act.columns:
    if column in not_denormal:
        continue
    elif column not in counter.keys():
        name = "else"
    else:
        name = column
    min0, min1 = sorted(tr_act[column].unique())[:2]
    r_min0, r_min1 = counter[name]["min"]
    counter[name]['s'] = (r_min1 - r_min0) / (min1 - min0)
    counter[name]['m'] = r_min0 - counter[name]['s']*min0
    t_sample = (tr_act[column]*counter[name]['s'] + counter[name]['m']).round(0).astype('int64')
    tr_act[column] = t_sample
    te_act[column] = (te_act[column]*counter[name]['s'] + counter[name]['m']).round(0).astype('int64')
    sorted_sample = sorted(t_sample.unique(), reverse=False)
    for_tap = '\t'*(4 - len(column)//8)
    for_print = "{}{}{}\t{}".format(column, for_tap, sorted_sample[:3], sorted_sample[-3:])
    print(for_print)

play_time			[0, 1, 2]	[587569, 587605, 604800]
npc_exp				[0, 1, 2]	[963788, 1067864, 1080468]
npc_hongmun			[0, 6, 9]	[99335137, 102329832, 107728899]
quest_exp			[0, 70, 80]	[20756180, 20792820, 20949700]
quest_hongmun			[0, 70, 130]	[151691388, 157509000, 227177750]
item_hongmun			[0, 10000, 20000]	[237100000, 252760000, 299060000]
game_combat_time		[0, 1, 2]	[402913, 404569, 404811]
get_money			[0, 1, 2]	[68135371955, 75256291607, 75755537696]
duel_cnt			[0, 1, 2]	[8991, 9268, 9578]
duel_win			[0, 1, 2]	[4436, 4437, 4491]
partybattle_cnt			[0, 1, 2]	[305, 321, 364]
partybattle_win			[0, 1, 2]	[177, 178, 232]
cnt_enter_inzone_solo		[0, 1, 2]	[1872, 1891, 2948]
cnt_enter_inzone_light		[0, 1, 2]	[613, 614, 620]
cnt_enter_inzone_skilled	[0, 1, 2]	[112, 133, 139]
cnt_enter_inzone_normal		[0, 1, 2]	[613, 614, 620]
cnt_enter_raid			[0, 1, 2]	[30, 36, 37]
cnt_enter_raid_light		[0, 1, 2]	[36, 103, 123]
cnt_enter_bam			[0, 1, 2]	[30, 41, 57]
cnt_clear_inzone_solo		[0, 1, 2]	[1860, 1874, 2939

In [8]:
tr_act.head()

,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt
0,7,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,4,278089,152118,36048,9111460,16556000,0,62088,...,0,0,0,0,0,0,0,733,0,0
1,8,3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...,5,337200,163768,54300,12313740,24834000,0,80692,...,0,0,0,0,0,0,0,1029,0,0
2,3,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,1228,28,0,8610,0,0,111,...,0,0,0,0,0,0,0,1,0,0
3,4,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,2,347,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,b8856358ff62e596fa07e3e40b8e7fd4b7729263c72b44...,4,701,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


블레이드&소울의 화폐는 10,000 동 = 100 은 = 1 금 으로 이뤄져 있다.  
실제 통용 단위는 금이기 때문에 금 단위로 바꿔준다.

In [9]:
tr_act['get_money'] /= 10000
te_act['get_money'] /= 10000
tr_act = tr_act.round({'get_money': 4})
te_act = te_act.round({'get_money': 4})

시간의 경우 분 단위로 조정한다.

In [10]:
tr_act['play_time'] /= 60
te_act['play_time'] /= 60
tr_act['game_combat_time'] /= 60
te_act['game_combat_time'] /= 60

# acc_id, wk 기준으로 정렬한다.
tr_act = tr_act.sort_values(by=['acc_id','wk']).reset_index(drop=True)
te_act = te_act.sort_values(by=['acc_id','wk']).reset_index(drop=True)
tr_act.head()

,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt
0,7,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,5,5671.650000,173339,31434,10267620,16640780,0,1426.383333,...,0,0,0,0,0,0,0,430,0,0
1,8,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,8783.600000,238297,52097,17142400,34621610,0,2261.066667,...,0,0,0,0,0,0,0,739,0,0
2,1,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.200000,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,2,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.366667,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,3,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,2,0.800000,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0


# activity 데이터 복원 완료

In [11]:
tr_act.to_csv('train_activity_pre.csv', index=False)
te_act.to_csv('test_activity_pre.csv', index=False)

# trade 데이터 복원
#### trade 데이터는 중복되는 데이터가 아주 많이 있다.
#### 이는 같은 타입인 다른 종류의 아이템을 동시에 거래하였다고 볼 수도 있지만,
#### money는 동시에 같은 양을 나눠 거래할 수 없으므로
#### log 상의 중복으로 생각하고 제거한다.

In [12]:
tr_trade.drop_duplicates(inplace=True)
te_trade.drop_duplicates(inplace=True)

In [13]:
tr_trade.columns

Index(['trade_week', 'trade_day', 'trade_time', 'source_acc_id',
       'target_acc_id', 'item_type', 'item_amount'],
      dtype='object')

#### 복원에 사용할 수 있는 아이템의 종류를 정하기 위해 min값, max값을 확인한다.
#### min 실제 값 = 1, max 실제 값 = 8 (거래 창의 크기 = 8칸)인 경우가 적합하다.
#### 게임에서 gem과 costume은 겹쳐지지 않으므로 조건에 부합한다.
* train 데이터는 gem, costume 사용 가능하다.  
* test 데이터는 gem 만 사용 가능하다.  
* train, test 데이터의 표준편차, 평균은 동일하므로 gem을 복원에 사용하기로 결정한다.

In [14]:
tr_minw, tr_maxw = map(sorted(tr_trade[tr_trade['item_type']=='weapon']['item_amount']).__getitem__, [0, -1])
tr_ming, tr_maxg = map(sorted(tr_trade[tr_trade['item_type']=='gem']['item_amount']).__getitem__, [0, -1])
tr_mina, tr_maxa = map(sorted(tr_trade[tr_trade['item_type']=='accessory']['item_amount']).__getitem__, [0, -1])
tr_minc, tr_maxc = map(sorted(tr_trade[tr_trade['item_type']=='costume']['item_amount']).__getitem__, [0, -1])
print("""
Weapon   \tMin Y: {:.8}\tMax Y: {:.8}
Gem      \tMin Y: {:.8}\tMax Y: {:.8}
Accessory\tMin Y: {:.8}\tMax Y: {:.8}
Costume  \tMin Y: {:.8}\tMax Y: {:.8}
""".format(tr_minw, tr_maxw, tr_ming, tr_maxg, tr_mina, tr_maxa, tr_minc, tr_maxc))


Weapon   	Min Y: -0.056345911	Max Y: -0.056345468
Gem      	Min Y: -0.056345911	Max Y: -0.05634585
Accessory	Min Y: -0.056345911	Max Y: -0.056345833
Costume  	Min Y: -0.056345911	Max Y: -0.05634585



In [15]:
te_minw, te_maxw = map(sorted(te_trade[te_trade['item_type']=='weapon']['item_amount']).__getitem__, [0, -1])
te_ming, te_maxg = map(sorted(te_trade[te_trade['item_type']=='gem']['item_amount']).__getitem__, [0, -1])
te_mina, te_maxa = map(sorted(te_trade[te_trade['item_type']=='accessory']['item_amount']).__getitem__, [0, -1])
te_minc, te_maxc = map(sorted(te_trade[te_trade['item_type']=='costume']['item_amount']).__getitem__, [0, -1])
print("""
Weapon   \tMin Y: {:.8}\tMax Y: {:.8}
Gem      \tMin Y: {:.8}\tMax Y: {:.8}
Accessory\tMin Y: {:.8}\tMax Y: {:.8}
Costume  \tMin Y: {:.8}\tMax Y: {:.8}
""".format(te_minw, te_maxw, te_ming, te_maxg, te_mina, te_maxa, te_minc, te_maxc))


Weapon   	Min Y: -0.056345911	Max Y: -0.056345867
Gem      	Min Y: -0.056345911	Max Y: -0.05634585
Accessory	Min Y: -0.056345911	Max Y: -0.056345867
Costume  	Min Y: -0.056345911	Max Y: -0.056345876



In [16]:
# activity 데이터 복원과 같은 방법으로 복원을 진행한다.
s = 7/(tr_maxg - tr_ming)
m = 1 - s*tr_ming
print("Standard Deviation: {}\tMean: {}".format(s, m))

Standard Deviation: 115117619.04653603	Mean: 6486408.0831658


In [17]:
tr_trade['item_amount'] = tr_trade['item_amount'] * s + m
te_trade['item_amount'] = te_trade['item_amount'] * s + m

블레이드&소울의 화폐는 10,000 동 = 100 은 = 1 금 으로 이뤄져 있다.  
실제 통용 단위는 금이기 때문에 금 단위로 바꿔준다.

In [18]:
tr_trade.loc[tr_trade['item_type']=='money', 'item_amount'] /= 10000
te_trade.loc[te_trade['item_type']=='money', 'item_amount'] /= 10000
tr_trade = tr_trade.sort_values(by=list(tr_trade.columns)).reset_index(drop=True).round({'item_amount': 4})
te_trade = te_trade.sort_values(by=list(te_trade.columns)).reset_index(drop=True).round({'item_amount': 4})
tr_trade.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,1,1,00:00:00,3c3baeef181bd1248e182851c47a39d16c7c343fd38ddc...,f9fd4dae996c583b6173d0dd5816db49fc03b09e9a532a...,grocery,2.0000
1,1,1,00:00:00,3c3baeef181bd1248e182851c47a39d16c7c343fd38ddc...,f9fd4dae996c583b6173d0dd5816db49fc03b09e9a532a...,money,28.5122
2,1,1,00:00:02,918c5389ed6f6e7becfa06e179966c12a2bfbd5b147a6d...,065a4d4a8607317210dd1283d0f5d2d64723736fae0d14...,money,200.0000
3,1,1,00:00:02,9c8c6cf69f6b2c48e7c377bb8a0327565973651bc8dbde...,0eb20d13ef580d1ccf04d264d69e0cca19e50dc7400926...,grocery,1.0000
4,1,1,00:00:02,9c8c6cf69f6b2c48e7c377bb8a0327565973651bc8dbde...,0eb20d13ef580d1ccf04d264d69e0cca19e50dc7400926...,money,28.5134


# trade 데이터 복원 완료

In [19]:
tr_trade.to_csv('train_trade_pre.csv', index=False)
te_trade.to_csv('test_trade_pre.csv', index=False)

# payment 데이터 복원, activity 복원 과정과 동일하다.

In [20]:
min0, min1 = sorted(tr_payment['payment_amount'].unique())[:2]
print("0: {}\n1: {}".format(min0, min1))

r_min0, r_min1 = 0, 990
s = (r_min1 - r_min0)/(min1 - min0)
m = r_min0 - s*min0
print("s: {}\nm: {}".format(s, m))

tr_sample = tr_payment['payment_amount']*s + m
sorted(tr_sample.unique(),reverse=False)[:10]

0: -0.149898470874251
1: -0.114742210890344
s: 28159.99200293713
m: 4221.139741071413


[0.0,
 989.9999999999995,
 1979.9999999999936,
 2969.999999999988,
 3959.9999999999854,
 4949.999999999982,
 5939.999999999979,
 6929.999999999973,
 7919.99999999997,
 8909.99999999997]

In [21]:
tr_payment['payment_amount'] = (tr_payment['payment_amount']*s + m).round(0).astype('int64')
te_payment['payment_amount'] = (te_payment['payment_amount']*s + m).round(0).astype('int64')
tr_payment = tr_payment.sort_values(by=['acc_id','payment_week']).reset_index(drop=True)
te_payment = te_payment.sort_values(by=['acc_id','payment_week']).reset_index(drop=True)
tr_payment.head()

,payment_week,acc_id,payment_amount
0,1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,0
1,2,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,0
2,3,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,0
3,4,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,0
4,5,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,0


# payment 데이터 복원 완료

In [22]:
tr_payment.to_csv('train_payment_pre.csv', index=False)
te_payment.to_csv('test_payment_pre.csv', index=False)